In [ ]:
import random
import math

# ==========================================
# 0. EL MOTOR MATEMÁTICO (REEMPLAZANDO NUMPY)
# ==========================================

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

